In [62]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from tqdm import tqdm_notebook as tqdm
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

#绘图设置
styles = ["white","dark",'whitegrid',"darkgrid"]
contexts = ['paper','talk','poster','notebook']
sns.set_context(contexts[1])
sns.set_style(styles[2])
tableau20 = np.array([(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)])/255.

Populating the interactive namespace from numpy and matplotlib


In [81]:
rnanames = ['miRNA', 'piRNA', 'Y_RNA', 'snRNA','srpRNA','tRNA',
            'lncRNA','mRNA','hg38other','nonhuman_N']
rna_ratio = pd.read_table('/home/student3/llq/plot_making/pie_plot/merged_pieplot_making.csv',sep = ' ',index_col=0)
np.array(rnanames)

array(['miRNA', 'piRNA', 'Y_RNA', 'snRNA', 'srpRNA', 'tRNA', 'lncRNA',
       'mRNA', 'hg38other', 'nonhuman_N'], dtype='<U10')

In [72]:
rna_ratio

,type,Sample_1S10,Sample_1S11,Sample_1S12,Sample_1S13,Sample_1S14,Sample_1S15,Sample_1S16,Sample_1S17,Sample_1S18,...,Sample_PC5,Sample_PC6,Sample_PC7,Sample_PC8,Sample_PC9,Sample_N13,Sample_N19,Sample_N1,Sample_N25,Sample_N7
0,miRNA,2344713,2703185,2845495,2403853,1867128,1441732,2375036,2428861,1609241,...,2703903,2997784,1757344,2380145,2755247,2670755,2490946,2848822,1988626,2307537
1,piRNA,2816805,3302901,3218331,4268506,4871759,3989122,3917185,4638096,3041241,...,2894462,3205091,1546672,2750730,2476995,2750297,2977610,3863280,4797211,2901460
2,Y_RNA,88835,127497,145142,90106,105377,87513,121418,144229,99332,...,115908,130839,110328,100963,125241,232866,162239,176945,88226,189700
3,srpRNA,4706,6594,9020,4990,6203,4299,5344,8500,5269,...,10108,8704,4850,5580,5501,4193,6065,6112,5640,6151
4,tRNA,332556,376539,473974,470160,581740,451309,440644,560626,351976,...,357776,366369,193189,361039,279649,292750,382350,379026,538845,323671
5,snRNA,29122,43159,58263,41155,46119,28647,37586,57221,38062,...,91110,78246,57954,53383,54441,33077,38814,55911,40017,60766
6,snoRNA,1779,2424,3768,2609,10770,1862,2523,3544,2443,...,3972,3339,2689,2643,2411,2155,2302,3604,2575,2972
7,lncRNA,26733,38346,35639,25523,31489,21601,26915,37259,28952,...,49285,44738,36209,37819,33028,29440,33668,40077,28202,37031
8,mRNA,51080,75211,90614,40932,47424,34856,46901,71520,55873,...,113917,90941,71234,58408,58025,50322,63354,54501,46379,58737
9,tucp,84492,122205,136251,92331,108782,74765,95228,126570,92871,...,152762,147094,90095,107706,102612,78919,100816,95735,98718,106634


In [75]:
def plot_pie(data, rnanames):
    '''
    data: table_ratio
    rnanames: rna type names
    adjustment: merge RNA with small percent together
    '''
    from bokeh.io import output_file, show
    from bokeh.palettes import Category20c
    from bokeh.plotting import figure
    from bokeh.transform import cumsum
    x = np.array(rnanames)
    y = np.array(data.loc[:,x].mean())
    z = np.array([float('{:.10f}'.format(y[i])) for i in range(y.shape[0])])*100
    labels = rnanames
    dataframe = pd.DataFrame(np.concatenate((x.reshape(-1,1),z.reshape(-1,1)),axis=1))
    dataframe.columns=['rna','percent']
    dataframe["percent"] = pd.to_numeric(dataframe["percent"])
    dataframe['angle'] = dataframe['percent']/dataframe['percent'].sum() * 2*pi
    dataframe['color'] = Category20c[len(x)]
    p = figure(plot_height=400, title="Pie Chart", toolbar_location=None,
               tools="hover", tooltips="@rna: @percent", x_range=(-0.5, 1.0))
    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend="rna", source=dataframe)
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    show(p)

In [76]:
plot_pie(rna_ratio,rnanames)

ValueError: only 2 non-keyword arguments accepted